In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageOps, ExifTags
from typing import Optional, Tuple, List
import matplotlib.patches as patches
import json
import os
import shutil

In [95]:
df2 = pd.read_csv('../data/VNIIGAZ_data/dataset_2/_annotations.csv')
df3 = pd.read_csv('../data/VNIIGAZ_data/dataset_3/_annotations.csv')
df4 = pd.read_csv('../data/VNIIGAZ_data/dataset_4/_annotations.csv')

In [96]:
print(df2.shape[0])
print(df3.shape[0])
print(df4.shape[0])
print(df2.shape[0]+df3.shape[0]+df4.shape[0])

3425
6867
30638
40930


In [97]:
merged_df = pd.concat([df2, df3, df4], ignore_index=True)

In [98]:
def convert_bbox_to_yolo_format(row):
    x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']
    x_center = (x_min + x_max) / 2 / row['width']
    y_center = (y_min + y_max) / 2 / row['height']
    width = (x_max - x_min) / row['width']
    height = (y_max - y_min) / row['height']

    return ' '.join(map(str, [x_center, y_center, width, height]))

In [99]:
merged_df['location'] = merged_df.apply(lambda x: convert_bbox_to_yolo_format(x), axis=1)

In [100]:
merged_df = merged_df[['filename', 'class', 'location']]

In [101]:
unique_photos = merged_df['filename'].unique()

In [107]:
train_photos, val_photos = train_test_split(unique_photos, test_size=0.1, random_state=20)
print(train_photos.shape)
print(val_photos.shape)

(5903,)
(656,)


In [111]:
def move_photos_labels(df, photo_names, source_path, dest_path, stage):
    for name in photo_names:
        name_df = df[df['filename'] == name]
        info = []
        for _, (name, cls, location) in name_df.iterrows():
            shutil.copy(os.path.join(source_path, name), os.path.join(dest_path, f'images/{stage}'))
            info.append(f'{cls} {location}')
        with open(os.path.join(dest_path, f'labels/{stage}/{name[:-4]}.txt'), 'a') as file:
            file.write('\n'.join(info))

In [112]:
source_path = '../data/all_images_second_part'
dest_path = '../data/second_part_data'

In [113]:
move_photos_labels(merged_df, train_photos, source_path, dest_path, 'train')

In [114]:
move_photos_labels(merged_df, val_photos, source_path, dest_path, 'val')

In [ ]:
import files